In [ ]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

In [21]:
param = {'lr'         : 0.01, # Coef to multiply gradients
         'momentum'   : 0.5,  # SGD momentum (default: 0.5) - extra term in descent
         'batch_size' : 512, # number of data samples to consider at once for training
         'epochs'     : 20,   #The number of Epochs is the number of times you go through the full dataset.

         }

train_dataset = datasets.MNIST('./data',
                               train=True,
                               download=True,
                               transform=transforms.ToTensor())

validation_dataset = datasets.MNIST('./data',
                                    train=False,
                                    transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=param['batch_size'],
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                                batch_size=param['batch_size'],
                                                shuffle=False)

In [ ]:
# Check structure and dimension of data

for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break



In [ ]:
# Plot images

pltsize=1
plt.figure(figsize=(10*pltsize, pltsize))

for i in range(10):
    plt.subplot(1,10,i+1)
    plt.axis('off')
    plt.imshow(X_train[i,:,:,:].numpy().reshape(28,28), cmap="gray_r")
    plt.title('Class: '+str(y_train[i].item()))



In [ ]:
# Define the architecture

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() #
        self.fc1 = nn.Linear(28*28, 50)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 50)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        return F.log_softmax(self.fc3(x), dim=1)

model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(),
                            param['lr'],
                            param['momentum'])

criterion = nn.CrossEntropyLoss() # cross-entropy meausers probability

print(model)



In [17]:
def train(epoch, log_interval=200):
    # Set model to training mode
    model.train()

    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad()

        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()

        # Update weights
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))



In [18]:
def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)

    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))

In [ ]:
%%time

lossv, accv = [], []
for epoch in range(1, param['epochs'] + 1):
    train(epoch)
    validate(lossv, accv)



In [ ]:
# Plot validation data and accuracy

plt.figure(figsize=(5,3))
plt.plot(np.arange(1,param['epochs']+1), lossv)
plt.title('validation loss')

plt.figure(figsize=(5,3))
plt.plot(np.arange(1,param['epochs']+1), accv)
plt.title('validation accuracy');

In [ ]:
# Number of images to evaluate
num_images_to_evaluate = 5

# Counter for evaluated images
evaluated_images = 0

# Iterate through the validation dataset
for batch_idx, (data, target) in enumerate(validation_loader):
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()

    # Forward pass
    output = model(data)

    # Get predicted labels
    predicted = output.argmax(dim=1)

    # Check if the current image is misclassified
    for idx in range(len(target)):
        if evaluated_images >= num_images_to_evaluate:
            break

        if predicted[idx] != target[idx]:
            # Print the evaluation for the misclassified image
            print(f"Image {evaluated_images + 1} - Predicted: {predicted[idx]}, Actual: {target[idx]}")

            # Plot the misclassified digit
            plt.imshow(data[idx].cpu().numpy().squeeze(), cmap='gray')
            plt.title(f"Predicted: {predicted[idx]}, Actual: {target[idx]}")
            plt.show()

            evaluated_images += 1

    if evaluated_images >= num_images_to_evaluate:
        break